<a href="https://colab.research.google.com/github/ahmedhesham47/Systems-Biology/blob/main/Systems_Networks_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Packages**

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# **Importing Files**

In [27]:
# pd1_network = pd.read_csv('PD_1_network.tsv', sep='\t')
# pd1_degree = pd.read_csv('PD_1_degree.tsv', sep='\t')
# pd1_pathways = pd.read_csv('PD_1_RCTM.tsv', sep='\t')

# pdL1_network = pd.read_csv('PD_L1_network.tsv', sep='\t')
# pdL1_degree = pd.read_csv('PD_L1_degree.tsv', sep='\t')
# pdL1_pathways = pd.read_csv('PD_L1_RCTM.tsv', sep='\t')

# ctla4_network = pd.read_csv('CTLA_4_network.tsv', sep='\t')
# ctla4_degree = pd.read_csv('CTLA_4_degree.tsv', sep='\t')
# ctla4_pathways = pd.read_csv('CTLA_4_RCTM.tsv', sep='\t')

# cd80_network = pd.read_csv('CD80_network.tsv', sep='\t')
# cd80_degree = pd.read_csv('CD80_degree.tsv', sep='\t')
# cd80_pathways = pd.read_csv('CD80_RCTM.tsv', sep='\t')

# mrna = pd.read_csv('mrna_cont.tsv', sep='\t')
# labels = pd.read_csv('labels_df.tsv', sep='\t')

# ravi_mrna = pd.read_csv('filtered_expression_ravi.tsv', sep='\t')
# labels_ravi = pd.read_csv('labels_ravi.tsv', sep='\t')

lui_0 = pd.read_csv('lui_0_filtered.tsv', sep='\t')
lui_1 = pd.read_csv('lui_1_filtered.tsv', sep='\t')
lui_res_network = pd.read_csv('lui_regulatory_network_responders.tsv', sep='\t')
lui_non_res_network = pd.read_csv('lui_regulatory_network_non_responders.tsv', sep='\t')
nodes_res = pd.read_csv('nodes_responders_lui.csv', sep=',')
nodes_non_res = pd.read_csv('nodes_nonresponders_lui.csv', sep=',')

In [ ]:
pd1_network = pd1_network[['#node1', 'node2']]
pdL1_network = pdL1_network[['#node1', 'node2']]
ctla4_network = ctla4_network[['#node1', 'node2']]
cd80_network = cd80_network[['#node1', 'node2']]

pd1_degree = pd1_degree[['#node', 'node_degree']]
pdL1_degree = pdL1_degree[['#node', 'node_degree']]
ctla4_degree = ctla4_degree[['#node', 'node_degree']]
cd80_degree = cd80_degree[['#node', 'node_degree']]

# **Helper Functions**

In [3]:
def find_common_interactions(list_of_dataframes):
    tuple_sets = []

    for df in list_of_dataframes:
        tuples = {tuple(sorted(row)) for row in df.to_numpy()}
        tuple_sets.append(tuples)

    common_tuples = set.intersection(*tuple_sets)

    return list(common_tuples)

In [4]:
def find_common_in_col(list_of_dataframes, column_name):
    columns = []

    for df in list_of_dataframes:
        if column_name in df.columns:
            columns.append(set(df[column_name]))

    common_values = set.intersection(*columns)

    return list(common_values)

In [35]:
def find_common_hubs(list_of_dataframes, degree_col_name, node_col_name, n_top=10):

    top_nodes_sets = []

    for df in list_of_dataframes:
        sorted_df = df.sort_values(by=degree_col_name, ascending=False).head(n_top)
        top_nodes = set(sorted_df[node_col_name])
        top_nodes_sets.append(top_nodes)

    common_nodes = set.intersection(*top_nodes_sets)

    return list(common_nodes)

In [6]:
def split_and_transpose(expression_df, labels_df, label_column_name):
  expression_df = expression_df.set_index('Sample Identifier')
  labels_df = labels_df.set_index('Sample Identifier')

  expression_1 = expression_df.loc[labels_df['ICB Response'] == 1.0].transpose().reset_index()
  expression_1.columns.name = None
  expression_1 = expression_1.rename(columns={'index': 'Gene'})

  expression_0 = expression_df.loc[labels_df['ICB Response'] == 0.0].transpose().reset_index()
  expression_0.columns.name = None
  expression_0 = expression_0.rename(columns={'index': 'Gene'})

  return expression_1, expression_0

# **Comparing PD-1, PD-L1, CTLA-4, CD80 networks**

In [ ]:
common_inrxns = find_common_interactions([pd1_network, pdL1_network, ctla4_network, cd80_network])

print(len(common_inrxns))
print(common_inrxns)

common_df = pd.DataFrame(common_inrxns, columns=['Node1', 'Node2'])
common_df.to_csv('common_interactions.tsv', index=False, sep='\t')

In [ ]:
common_nodes = find_common_in_col([pd1_degree, pdL1_degree, ctla4_degree, cd80_degree], '#node')

print(len(common_nodes))
print(common_nodes)

common_nodes_df = pd.DataFrame(common_nodes, columns=['Common_nodes'])
common_nodes_df.to_csv('common_nodes.tsv', index=False, sep='\t')

In [ ]:
common_pathways = find_common_in_col([pd1_pathways, pdL1_pathways, ctla4_pathways, cd80_pathways], 'term description')

print(len(common_pathways))
print(common_pathways)

common_pathways_df = pd.DataFrame(common_pathways, columns=['Common_pathways'])
common_pathways_df.to_csv('common_pathways.tsv', index=False, sep='\t')

In [ ]:
common_hubs = find_common_hubs([pd1_degree, pdL1_degree, ctla4_degree, cd80_degree])

print(len(common_hubs))
print(common_hubs)

common_hubs_df = pd.DataFrame(common_hubs, columns=['Common_hubs'])
common_pathways_df.to_csv('common_hubs.tsv', index=False, sep='\t')

# **Pre-processing and filtering Lui (cBioPortal) gene expression dataset to be ready for GENIE3**

In [ ]:
lui_responders = pd.read_csv('expression_1.tsv', sep='\t', index_col=0)
lui_non_responders = pd.read_csv('expression_0.tsv', sep='\t', index_col=0)
cytokine_genes = pd.read_csv('cytokine_genes.txt', sep='\t', header=0)
t_cell_genes = pd.read_csv('T cell genes.txt', sep='\t', header=0)
cytokine_genes = list(cytokine_genes['Symbol'])
t_cell_genes = list(t_cell_genes['Symbol'])
unique_union = set(cytokine_genes + t_cell_genes)

In [ ]:
lui_responders = lui_responders.loc[lui_responders.index.isin(unique_union)]

lui_non_responders = lui_non_responders.loc[lui_non_responders.index.isin(unique_union)]

lui_responders.to_csv('lui_1_filtered.tsv', sep='\t')
lui_non_responders.to_csv('lui_0_filtered.tsv', sep='\t')

# **Comparing GRNs between ICB responders and non-responders**

In [ ]:
# lui_res_network
lui_non_res_network

plt.figure(figsize=(10, 6))

sns.histplot(lui_res_network['weight'], color="blue", label='Responders Weights Distribution', kde=True)
sns.histplot(lui_non_res_network['weight'], color="red", label='Non-Responders Weights Distribution', kde=True)

plt.title('Distribution of Weight Values')
plt.legend()

plt.savefig('weights_distribution.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
lui_res_network

common_inrxns = find_common_interactions([lui_res_network.iloc[:, :-1], lui_non_res_network.iloc[:, :-1]])

print(len(common_inrxns))
print(common_inrxns)

In [ ]:
common_nodes = find_common_in_col([nodes_res, nodes_non_res], 'name')

print(len(common_nodes))
print(common_nodes)

In [ ]:
common_hubs = find_common_hubs([nodes_res, nodes_non_res], 'EdgeCount', 'name', 10)

print(len(common_hubs))
print(common_hubs)